In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'gabor'
CHANNEL = 'blue'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_gabor_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,38136.56,1.20,-0.21,0.43,0.11,-0.14,-0.12,-2.01,0.67,-0.42,...,-0.18,0.33,-0.02,0.69,-0.10,-0.44,0.19,0.03,0.03,0.04
1,1.20,16325.69,0.61,-0.43,-0.06,-0.10,-0.01,1.34,-0.21,-0.36,...,0.00,-0.00,0.01,0.88,-0.26,-0.12,-0.27,0.06,0.05,0.07
2,-0.21,0.61,6576.85,-0.11,0.00,0.07,0.04,0.11,0.48,-0.15,...,0.05,-0.04,0.03,-0.63,0.52,-0.02,-0.32,-0.01,0.09,-0.01
3,0.43,-0.43,-0.11,2797.89,-0.05,0.03,-0.02,-0.59,-0.21,0.00,...,-0.04,-0.05,0.02,0.33,-0.02,0.12,0.06,0.04,-0.04,-0.00
4,0.11,-0.06,0.00,-0.05,1152.86,-0.01,0.01,-0.02,0.00,0.22,...,0.00,0.01,0.01,0.01,0.13,-0.09,0.08,0.01,0.03,-0.02
5,-0.14,-0.10,0.07,0.03,-0.01,415.58,0.00,-0.26,0.09,-0.03,...,-0.02,-0.01,0.01,-0.15,0.01,0.01,0.04,0.00,0.00,-0.00
6,-0.12,-0.01,0.04,-0.02,0.01,0.00,279.82,0.06,0.00,0.08,...,0.02,-0.00,-0.01,-0.05,-0.01,0.02,-0.03,0.03,-0.01,0.00
7,-2.01,1.34,0.11,-0.59,-0.02,-0.26,0.06,54289.06,-0.10,0.51,...,-0.02,0.14,-0.13,0.42,-1.38,-0.39,-0.54,0.32,0.16,-0.13
8,0.67,-0.21,0.48,-0.21,0.00,0.09,0.00,-0.10,21607.08,0.32,...,-0.12,0.04,0.00,0.54,0.86,-0.70,0.20,0.16,-0.04,-0.03


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00002,-0.00003,-0.00004,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00002,-0.00000,-0.00003,0.00002,0.00000,0.00001,0.00002
1,0.00005,1.00000,0.00006,-0.00006,-0.00001,-0.00004,-0.00000,0.00005,-0.00001,-0.00003,...,0.00000,-0.00000,0.00000,0.00003,-0.00002,-0.00001,-0.00004,0.00001,0.00002,0.00004
2,-0.00001,0.00006,1.00000,-0.00003,0.00000,0.00004,0.00003,0.00001,0.00004,-0.00002,...,0.00002,-0.00002,0.00003,-0.00004,0.00005,-0.00000,-0.00007,-0.00000,0.00005,-0.00001
3,0.00004,-0.00006,-0.00003,1.00000,-0.00003,0.00002,-0.00002,-0.00005,-0.00003,0.00000,...,-0.00002,-0.00005,0.00003,0.00003,-0.00000,0.00003,0.00002,0.00002,-0.00003,-0.00000
4,0.00002,-0.00001,0.00000,-0.00003,1.00000,-0.00002,0.00001,-0.00000,0.00000,0.00007,...,0.00000,0.00002,0.00001,0.00000,0.00003,-0.00003,0.00004,0.00001,0.00005,-0.00004
5,-0.00003,-0.00004,0.00004,0.00002,-0.00002,1.00000,0.00001,-0.00005,0.00003,-0.00002,...,-0.00003,-0.00002,0.00003,-0.00003,0.00000,0.00001,0.00004,0.00000,0.00000,-0.00001
6,-0.00004,-0.00000,0.00003,-0.00002,0.00001,0.00001,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00002,-0.00001,-0.00001,0.00001,-0.00003,0.00005,-0.00003,0.00001
7,-0.00004,0.00005,0.00001,-0.00005,-0.00000,-0.00005,0.00002,1.00000,-0.00000,0.00002,...,-0.00000,0.00003,-0.00004,0.00001,-0.00004,-0.00002,-0.00004,0.00004,0.00003,-0.00004
8,0.00002,-0.00001,0.00004,-0.00003,0.00000,0.00003,0.00000,-0.00000,1.00000,0.00002,...,-0.00002,0.00001,0.00000,0.00002,0.00004,-0.00006,0.00002,0.00003,-0.00001,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

13.085013019410306

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.04512590e+07 4.96059085e+07 4.52195620e+07 3.54351705e+07
 3.39917104e+07 3.03337216e+07 1.11259536e+07 8.06261793e+06
 7.69834023e+06 6.39408804e+06 5.98319257e+06 5.31897610e+06
 1.92045647e+06 1.45655455e+06 1.22015571e+06 1.03240822e+06
 9.56559673e+05 8.49250427e+05 3.43827980e+05 2.30354882e+05
 2.12861257e+05 1.84581196e+05 1.66256937e+05 1.43297271e+05
 5.80248749e+04 3.98133379e+04 3.48261299e+04 3.19190944e+04
 3.08649228e+04 2.38322889e+04 8.63870231e+03 6.49488891e+03
 5.77775796e+03 5.09898256e+03 4.53528021e+03 3.39036559e+03
 2.20501342e+03 1.69090139e+03 1.27024754e+03 1.08358630e+03
 9.93684389e+02 5.29786484e-24]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.028794,-0.006680,-0.002489,-0.001059,-0.000431,-0.000160,-0.000103,0.995784,-0.009623,-0.003444,...,-0.000419,-0.000170,-0.000077,-0.047550,-0.007676,-0.002735,-0.001137,-0.000446,-0.000183,-0.000079
1,-0.047600,-0.006641,-0.002384,-0.001011,-0.000412,-0.000144,-0.000102,0.042420,-0.009937,-0.003313,...,-0.000393,-0.000157,-0.000070,-0.235833,-0.007641,-0.002582,-0.001065,-0.000439,-0.000178,-0.000069
2,-0.098349,-0.010570,-0.003764,-0.001561,-0.000643,-0.000234,-0.000157,0.051087,-0.016039,-0.005227,...,-0.000618,-0.000253,-0.000110,0.961571,-0.012262,-0.004093,-0.001699,-0.000668,-0.000279,-0.000117
3,0.723049,-0.000357,-0.000136,-0.000056,-0.000024,-0.000014,-0.000007,0.001141,-0.000580,-0.000213,...,-0.000027,-0.000002,-0.000003,0.002999,-0.000492,-0.000152,-0.000053,-0.000020,-0.000007,-0.000002
4,0.629790,-0.014499,-0.004906,-0.002014,-0.000826,-0.000299,-0.000207,0.035167,-0.023394,-0.006879,...,-0.000807,-0.000324,-0.000147,0.085958,-0.017085,-0.005387,-0.002185,-0.000877,-0.000362,-0.000150
5,0.256865,-0.023657,-0.007765,-0.003195,-0.001315,-0.000475,-0.000316,0.045134,-0.039456,-0.010959,...,-0.001285,-0.000527,-0.000225,0.094050,-0.028058,-0.008507,-0.003461,-0.001380,-0.000568,-0.000232
6,0.018726,-0.042291,-0.007795,-0.003075,-0.001244,-0.000444,-0.000302,0.010705,0.991627,-0.011745,...,-0.001216,-0.000494,-0.000214,0.014790,-0.069582,-0.008749,-0.003297,-0.001301,-0.000539,-0.000223
7,0.002173,-0.016867,-0.001512,-0.000555,-0.000249,-0.000083,-0.000054,0.001301,0.010572,-0.002324,...,-0.000235,-0.000086,-0.000040,0.001841,-0.629135,-0.001615,-0.000608,-0.000234,-0.000096,-0.000043
8,0.018197,-0.181305,-0.013096,-0.004933,-0.002002,-0.000717,-0.000483,0.011235,0.077602,-0.020861,...,-0.001952,-0.000778,-0.000350,0.014988,0.753411,-0.014779,-0.005332,-0.002102,-0.000856,-0.000358
9,0.007562,0.890045,-0.007069,-0.002648,-0.001054,-0.000388,-0.000255,0.004785,0.025158,-0.011865,...,-0.001030,-0.000418,-0.000183,0.006304,0.064972,-0.008111,-0.002853,-0.001106,-0.000449,-0.000183


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.0042164428991869896,
 0.032287148024461976,
 0.038428973269403754,
 0.27695146625565314,
 0.3363583607589513,
 0.08146639556406077,
 0.008373261695121315,
 0.22316966048957043,
 0.24658943010988377,
 0.10995453903442398,
 0.18306950751713158,
 0.1291463884255687,
 0.01190069792976578,
 0.026710828126624175,
 0.042546129926090304,
 0.09452842359902314,
 0.21982306984297373,
 0.22808072186429584,
 0.013918427284397517,
 0.09566422462729862,
 0.13565079300762473,
 0.11449996920415861,
 0.2710012075446252,
 0.3400949594499858,
 0.02243046214525779,
 0.058158143637865534,
 0.11081708549894276,
 0.19232111935077567,
 0.2549275120248018,
 0.4653132498609227,
 0.04207233558638335,
 0.11292099354825191,
 0.17852489182634224,
 0.28152218238042537,
 0.28378386555426116,
 0.37428694999257006,
 0.17445609322239586,
 0.2323627957568537,
 0.21435775870380502,
 0.28885033832307516,
 0.19609209307325037,
 0.5788978775049001]